In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
conda install -c conda-forge fbprophet

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - rapidsai/linux-64::libcuml==21.12.00=cuda11_g04c4927f3_0
  - conda-forge/linux-64::abseil-cpp==20211102.0=h93e1e8c_3
  - rapidsai/linux-64::dask-cudf==21.12.02=cuda_11_py37_g06540b9b37_0
  - conda-forge/linux-64::pyarrow==5.0.0=py37h8cf84b7_35_cuda
  - rapidsai/linux-64::cuml==21.12.00=cuda11_py37_g04c4927f3_0
  - conda-forge/linux-64::grpc-cpp==1.45.2=he70e3f0_3
  - rapidsai/linux-64::libcudf==21.12.02=cuda11_g06540b9b37_0
  - conda-forge/linux-64::arrow-cpp==5.0.0=py37h846d386_35_cuda
  - rapidsai/linux-64::cudf==21.12.02=cuda_11_py37_g06540b9b37_0
  - conda-forge/noarch::parquet-cpp==1.5.1=2
  - conda-forge/linux-64::libabseil==20211102.0=cxx17_h48a1fff_3
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solvi

In [5]:
# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from operator import attrgetter
#!pip install pmdarima
from pmdarima.arima import auto_arima
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
#!pip install pystan
#!conda install -c conda-forge fbprophet
#!pip install fbprophet

from fbprophet import Prophet

In [16]:
import pandas as pd

# Read the dataset into a pandas dataframe
df = pd.read_csv('/kaggle/input/ciscodataset/PLID_Bookings_Problem1.csv')

# Drop the 'Product Family' column as it is not needed for the analysis
df.drop('Product Family', axis=1, inplace=True)

# Split 'Fiscal Quarter' into 'Quarter' and 'Year' columns
df[['Quarter', 'Year']] = df['Fiscal Quarter'].str.split(' ', expand=True)

# Drop the 'Fiscal Quarter' column as it is no longer needed
df.drop('Fiscal Quarter', axis=1, inplace=True)

# Extract the month from the 'Fiscal Month' column and create a new column called 'Month'
df['Month'] = df['Fiscal Month'].str.extract('([A-Z]+)', expand=False)

# Drop the 'Fiscal Month' column as it is no longer needed
df.drop('Fiscal Month', axis=1, inplace=True)

# Convert the 'Booking_Date' column to a pandas datetime object
df['Booking_Date'] = pd.to_datetime(df['Booking_Date'], format='%d-%m-%Y')

# Print the cleaned dataframe
print(df.head())


  Business Unit             PLID  Booked_Qty Booking_Date Quarter    Year  \
0         CSWBU  CBR-D121-DS-MOD         204   2016-08-01      Q1  FY2022   
1         CSWBU  CBR-D121-DS-MOD         564   2016-10-01      Q1  FY2022   
2         CSWBU  CBR-D121-DS-MOD        1476   2016-09-01      Q1  FY2022   
3         CSWBU  CBR-D121-DS-MOD         374   2017-08-01      Q1  FY2002   
4         CSWBU  CBR-D121-DS-MOD        6094   2017-10-01      Q1  FY2002   

  Month  
0   AUG  
1   OCT  
2   SEP  
3   AUG  
4   OCT  


In [17]:
df['Booking_Date'] = pd.to_datetime(df['Booking_Date'])
df['Booking_Month'] = df['Booking_Date'].dt.month
df['Booking_Year'] = df['Booking_Date'].dt.year

In [18]:
df['Fiscal_q']= '' + df['Quarter'].astype(str) + 'FY' + df['Booking_Date'].dt.year.astype(str)


In [19]:
df

,Business Unit,PLID,Booked_Qty,Booking_Date,Quarter,Year,Month,Booking_Month,Booking_Year,Fiscal_q
0,CSWBU,CBR-D121-DS-MOD,204,2016-08-01,Q1,FY2022,AUG,8,2016,Q1FY2016
1,CSWBU,CBR-D121-DS-MOD,564,2016-10-01,Q1,FY2022,OCT,10,2016,Q1FY2016
2,CSWBU,CBR-D121-DS-MOD,1476,2016-09-01,Q1,FY2022,SEP,9,2016,Q1FY2016
3,CSWBU,CBR-D121-DS-MOD,374,2017-08-01,Q1,FY2002,AUG,8,2017,Q1FY2017
4,CSWBU,CBR-D121-DS-MOD,6094,2017-10-01,Q1,FY2002,OCT,10,2017,Q1FY2017
...,...,...,...,...,...,...,...,...,...,...
10991,RTOBU,C9800-40-K9,229,2020-06-01,Q4,FY2011,JUN,6,2020,Q4FY2020
10992,RTOBU,C9800-40-K9,289,2020-05-01,Q4,FY2011,MAY,5,2020,Q4FY2020
10993,RTOBU,C9800-40-K9,1231,2021-07-01,Q4,FY2012,JUL,7,2021,Q4FY2021
10994,RTOBU,C9800-40-K9,591,2021-06-01,Q4,FY2012,JUN,6,2021,Q4FY2021


In [20]:
df['Booking_Date'] = df['Booking_Date'].dt.to_period('M')


In [21]:
df

,Business Unit,PLID,Booked_Qty,Booking_Date,Quarter,Year,Month,Booking_Month,Booking_Year,Fiscal_q
0,CSWBU,CBR-D121-DS-MOD,204,2016-08,Q1,FY2022,AUG,8,2016,Q1FY2016
1,CSWBU,CBR-D121-DS-MOD,564,2016-10,Q1,FY2022,OCT,10,2016,Q1FY2016
2,CSWBU,CBR-D121-DS-MOD,1476,2016-09,Q1,FY2022,SEP,9,2016,Q1FY2016
3,CSWBU,CBR-D121-DS-MOD,374,2017-08,Q1,FY2002,AUG,8,2017,Q1FY2017
4,CSWBU,CBR-D121-DS-MOD,6094,2017-10,Q1,FY2002,OCT,10,2017,Q1FY2017
...,...,...,...,...,...,...,...,...,...,...
10991,RTOBU,C9800-40-K9,229,2020-06,Q4,FY2011,JUN,6,2020,Q4FY2020
10992,RTOBU,C9800-40-K9,289,2020-05,Q4,FY2011,MAY,5,2020,Q4FY2020
10993,RTOBU,C9800-40-K9,1231,2021-07,Q4,FY2012,JUL,7,2021,Q4FY2021
10994,RTOBU,C9800-40-K9,591,2021-06,Q4,FY2012,JUN,6,2021,Q4FY2021


In [25]:
#prophet code for demad prediction
import pandas as pd
from fbprophet import Prophet


# Convert Booking_Date to datetime format
df['Booking_Date'] = pd.to_datetime(df['Booking_Date'])

# Prepare the data for Prophet
df = df.groupby(['PLID', 'Booking_Date'], as_index=False)['Booked_Qty'].sum()
df = df.rename(columns={'Booking_Date': 'ds', 'Booked_Qty': 'y'})



TypeError: Passing PeriodDtype data is invalid. Use `data.to_timestamp()` instead

In [23]:
df['PLID']

0        CBR-D121-DS-MOD
1        CBR-D121-DS-MOD
2        CBR-D121-DS-MOD
3        CBR-D121-DS-MOD
4        CBR-D121-DS-MOD
              ...       
10991        C9800-40-K9
10992        C9800-40-K9
10993        C9800-40-K9
10994        C9800-40-K9
10995        C9800-40-K9
Name: PLID, Length: 10996, dtype: object

In [24]:
# Create a Prophet model for each PLID
models = {}
for plid in df['PLID'].unique():
    plid_df = df[df['PLID'] == plid]
    model = Prophet()
    model.fit(plid_df)



ValueError: Dataframe must have columns "ds" and "y" with the dates and values respectively.

In [2]:
import pandas as pd

In [3]:
# Predict the demand for each PLID for the upcoming quarters
future = pd.DataFrame()
for plid, model in models.items():
    plid_future = model.make_future_dataframe(periods=3, freq='Q')
    plid_forecast = model.predict(plid_future)
    plid_forecast['PLID'] = plid
    future = future.append(plid_forecast, ignore_index=True)




NameError: name 'models' is not defined

In [43]:
# Read data
data = pd.read_csv("/kaggle/input/ciscodataset/PLID_Bookings_Problem1.csv")
data.dtypes

Business Unit     object
Product Family    object
PLID              object
Fiscal Quarter    object
Fiscal Month      object
Booked_Qty         int64
Booking_Date      object
dtype: object

In [44]:

data = data.rename(columns={'Business Unit':'Warehouse', 
                        'Product Family':'Product_Category', 
                        'PLID':'Product_Code', 
                        'Fiscal Quarter':'fiscal_quarter', 
                        'Fiscal Month':'fiscal_month', 
                        'Booked_Qty':'Order_Demand', 
                        'Booking_Date':'Date'})

In [45]:
# Format Date to datetime
data['Date'] = pd.to_datetime(data['Date'])


data['Order_Demand'] = pd.to_numeric(data['Order_Demand'])

# Drop warehouse and category columns
data = data.drop(columns=['Warehouse','Product_Category'])

# Sort data by period
data = data.sort_values('Date').reset_index().drop('index',axis=1)

In [46]:
data


,Product_Code,fiscal_quarter,fiscal_month,Order_Demand,Date
0,A9K1000-ESP100,Q2 FY2013,JAN FY2013,63,2013-01-01
1,SFP-10G-LR,Q2 FY2013,JAN FY2013,18921,2013-01-01
2,A9K1000-ESP100,Q3 FY2013,FEB FY2013,31,2013-01-02
3,SFP-10G-LR,Q3 FY2013,FEB FY2013,12810,2013-01-02
4,A9K1000-ESP100,Q3 FY2013,MAR FY2013,61,2013-01-03
...,...,...,...,...,...
10991,DN2-SD19T61X-EV,Q2 FY2022,DEC FY2022,3976,2022-01-12
10992,C9200L-48P-4G-E,Q2 FY2022,DEC FY2022,9963,2022-01-12
10993,VEDGE-2000-AC-K9,Q2 FY2022,DEC FY2022,76,2022-01-12
10994,N540-24Z8Q2C-SYS,Q2 FY2022,DEC FY2022,296,2022-01-12


In [47]:
print("Data Types")
print(data.dtypes)
print("\t")

print(data.head())
print("\t")

print("Data Dimension",data.shape)
print("\t")

print("The number of products is",len(data['Product_Code'].value_counts().index))
print("Period range is from",data['Date'].min(),"to", data['Date'].max())
print("Order Qty is from",data['Order_Demand'].min(),"to", data['Order_Demand'].max())
print("\t")

print('Number of missing values by column',[sum(data[i].isnull()) for i in data.columns])
print('All missing values are in Date column.')

Data Types
Product_Code              object
fiscal_quarter            object
fiscal_month              object
Order_Demand               int64
Date              datetime64[ns]
dtype: object
	
     Product_Code fiscal_quarter fiscal_month  Order_Demand       Date
0  A9K1000-ESP100      Q2 FY2013   JAN FY2013            63 2013-01-01
1      SFP-10G-LR      Q2 FY2013   JAN FY2013         18921 2013-01-01
2  A9K1000-ESP100      Q3 FY2013   FEB FY2013            31 2013-01-02
3      SFP-10G-LR      Q3 FY2013   FEB FY2013         12810 2013-01-02
4  A9K1000-ESP100      Q3 FY2013   MAR FY2013            61 2013-01-03
	
Data Dimension (10996, 5)
	
The number of products is 250
Period range is from 2013-01-01 00:00:00 to 2022-01-12 00:00:00
Order Qty is from 0 to 322807
	
Number of missing values by column [0, 0, 0, 0, 0]
All missing values are in Date column.


In [48]:
# Examine demand of several products
print("Demand of  SFP-10G-LR")
print(data.loc[data['Product_Code'] == ' SFP-10G-LR'].head(10))
print("\t")
print("Demand of A9K1000-ESP100")
print(data.loc[data['Product_Code'] == 'A9K1000-ESP100'].head(10))

Demand of  SFP-10G-LR
Empty DataFrame
Columns: [Product_Code, fiscal_quarter, fiscal_month, Order_Demand, Date]
Index: []
	
Demand of A9K1000-ESP100
      Product_Code fiscal_quarter fiscal_month  Order_Demand       Date
0   A9K1000-ESP100      Q2 FY2013   JAN FY2013            63 2013-01-01
2   A9K1000-ESP100      Q3 FY2013   FEB FY2013            31 2013-01-02
4   A9K1000-ESP100      Q3 FY2013   MAR FY2013            61 2013-01-03
6   A9K1000-ESP100      Q3 FY2013   APR FY2013           104 2013-01-04
11  A9K1000-ESP100      Q4 FY2013   MAY FY2013            19 2013-01-05
14  A9K1000-ESP100      Q4 FY2013   JUN FY2013            57 2013-01-06
16  A9K1000-ESP100      Q4 FY2013   JUL FY2013            59 2013-01-07
20  A9K1000-ESP100      Q1 FY2013   AUG FY2013            10 2013-01-08
22  A9K1000-ESP100      Q1 FY2013   SEP FY2013            16 2013-01-09
23  A9K1000-ESP100      Q1 FY2013   OCT FY2013           123 2013-01-10


In [49]:
data = data.drop(columns=['fiscal_quarter','fiscal_month'])


In [50]:
data

,Product_Code,Order_Demand,Date
0,A9K1000-ESP100,63,2013-01-01
1,SFP-10G-LR,18921,2013-01-01
2,A9K1000-ESP100,31,2013-01-02
3,SFP-10G-LR,12810,2013-01-02
4,A9K1000-ESP100,61,2013-01-03
...,...,...,...
10991,DN2-SD19T61X-EV,3976,2022-01-12
10992,C9200L-48P-4G-E,9963,2022-01-12
10993,VEDGE-2000-AC-K9,76,2022-01-12
10994,N540-24Z8Q2C-SYS,296,2022-01-12


In [51]:
### Steps to remove returns:
# This concerns Date values, thus observations with missing valuse in Date column need to be removed first.
# Observations with missing values are extracted into a new table. Investigation into missing values will be 
# performed later.
data_null = data.loc[data['Date'].isnull() == True,]
data = data.dropna(how='any',axis=0)
[sum(data[i].isnull()) for i in data.columns]

# # Create a column with absolute values of order_demand column
# data['Abs_Demand'] = abs(data['Order_Demand'])

# # Remove all rows that have the same products, dates, and absolute values of demand
# data_nodup = data.drop_duplicates(subset = ['Product_Code','Date','Abs_Demand'], keep = False)

[0, 0, 0]

In [52]:
# The number of products have null values
print("The number of products with null is",len(data_null['Product_Code'].value_counts().index))


The number of products with null is 0


In [53]:
data['Date'] = data['Date'].dt.to_period('M')
data = data.rename(columns = {"Date": 'Period'})
data = data.groupby(['Product_Code','Period'])['Order_Demand'].sum().reset_index().sort_values('Period').reset_index().drop('index',axis=1)
data.head()

,Product_Code,Period,Order_Demand
0,SFP-10G-LR,2013-01,274974
1,CPAK-100G-LR4,2013-01,87
2,ISR4451-X/K9,2013-01,167
3,A9K1000-ESP100,2013-01,751
4,ISR4351/K9,2014-01,9


In [54]:
full_period = pd.date_range('2013-01-01','2022-01-12', freq='MS').to_period('M')

full_period = set(full_period)
data_period = set(data['Period'])
full_period.difference(data_period)
# The missing periods are 5 months in 2011, including Feb, Mar, Apr, Jul, and Aug.
# There are various possible reasons for the missing periods: No demands are in these months, warehouses to be
# closed in these months for some reason, missing data in these periods, etc.
# To ensure that the training data will not be misleading, all data before Sep 2011 will be removed.
data = data.loc[data['Period'] > '2011-08']

In [55]:
data

,Product_Code,Period,Order_Demand
0,SFP-10G-LR,2013-01,274974
1,CPAK-100G-LR4,2013-01,87
2,ISR4451-X/K9,2013-01,167
3,A9K1000-ESP100,2013-01,751
4,ISR4351/K9,2014-01,9
...,...,...,...
1184,HX-CPU-I6258R,2022-01,856
1185,GLC-TE,2022-01,589334
1186,FPR4115-NGFW-K9,2022-01,1087
1187,DS-X9648-1536K9,2022-01,3650


In [56]:
# Criteria 2: Remove stopped products
latest_datamonth = data.groupby('Product_Code')['Period'].max().reset_index()
latest_datamonth = latest_datamonth.loc[latest_datamonth['Period'] > '2015-12']
data = data.loc[data['Product_Code'].isin(latest_datamonth['Product_Code'])]

In [57]:
duration_data = data.groupby('Product_Code').agg({'Period': ['min', 'max']}).reset_index()
duration_data['Duration'] = (duration_data[('Period', 'max')] - duration_data[('Period', 'min')]
                            ).apply(attrgetter('n')) + 1
duration_data = duration_data.loc[duration_data['Duration'] > 24 ]
data = data.loc[data['Product_Code'].isin(duration_data['Product_Code'])]

In [58]:
final_df = data.sort_values(by=['Order_Demand'], ascending=False)
x=[]
y=[]
for i in range(0,20):
    y.append(final_df['Product_Code'][i])
x.append(y)
print(x)

[['SFP-10G-LR', 'CPAK-100G-LR4', 'ISR4451-X/K9', 'A9K1000-ESP100', 'ISR4351/K9', 'ISR4431/K9', 'CP-7841-K9', 'ISR4321/K9', 'CPAK-100G-LR4', 'A9K1001-X', 'A9K1000-ESP100', 'N9K-C9508', 'QSFP-40G-SR-BD', 'SFP-10G-LR-S', 'ISR4451-X/K9', 'CP-8861-K9', 'SFP-10G-LR', 'CP-8811-K9', 'CP-7821-K9', 'WS-C3560CX-12PC-S']]


In [59]:
data_list =y


In [60]:
data_list

['SFP-10G-LR',
 'CPAK-100G-LR4',
 'ISR4451-X/K9',
 'A9K1000-ESP100',
 'ISR4351/K9',
 'ISR4431/K9',
 'CP-7841-K9',
 'ISR4321/K9',
 'CPAK-100G-LR4',
 'A9K1001-X',
 'A9K1000-ESP100',
 'N9K-C9508',
 'QSFP-40G-SR-BD',
 'SFP-10G-LR-S',
 'ISR4451-X/K9',
 'CP-8861-K9',
 'SFP-10G-LR',
 'CP-8811-K9',
 'CP-7821-K9',
 'WS-C3560CX-12PC-S']

In [61]:
# create data for forecasting
start = data['Period'][0] + 3
fcastperiods = 12  # forecast periods is subject to change by forecast users
full_period = [start + x for x in range(0,fcastperiods)]

In [62]:
# function to split train and test set
# history length of a product starts from the month of first order, not all products have the same history length
# the function is to split train-test based on product's history length instead of dataset length
def train_test(data):
    myList = data.tolist()
    i = myList.index(next(filter(lambda x: x!=0, myList)))
    data = data.iloc[i:,]
    train = data[:int(0.7*(len(data)))]
    test = data[int(0.7*len(data)):]
    return train, test, data



In [35]:
# data=data_list

In [63]:
# build function to run model for all columns
Arima = ['Arima']
ArimaFcastPerf = pd.DataFrame({'Models': Arima})
ArimaData = pd.DataFrame({'Period': full_period, 'Model': 'Arima'})



In [64]:
def arimafcast(data):
    for i in data:
        try:
            train, test, full = train_test(data[i])
            
            # Test model
            model_pred = auto_arima(train, start_p=0, start_q=0, max_p=3, max_q=3,
                                    m=1, seasonal=False, trace=True, 
                                    error_action='ignore', suppress_warnings=True)
            model_pred.fit(train)
            pred = np.round(model_pred.predict(n_periods=len(test)))
            ArimaFcastPerf[i] = sqrt(mean_squared_error(test,pred))
            
            # Forecast model
            model_fc = auto_arima(full, start_p=0, start_q=0, max_p=3, max_q=3,
                                    m=1, seasonal=False, trace=True, 
                                    error_action='ignore', suppress_warnings=True)
            model_fc.fit(full)
            forecast = np.round(model_fc.predict(n_periods=fcastperiods+2))
            ArimaData[i] = forecast[-fcastperiods:]
        except:
            ArimaFcastPerf[i] = np.nan
            ArimaData[i] = np.nan
    return ArimaFcastPerf, ArimaData


In [65]:
arimafcast(data)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=21776.399, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=21566.110, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.43 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=21774.401, Time=0.04 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=21474.285, Time=0.10 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=21433.002, Time=0.15 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=21265.425, Time=0.48 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=21264.445, Time=0.40 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=21263.997, Time=0.49 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.63 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.41 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=1.21 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=inf, Time=0.30 sec

Best model:  ARIMA(1,1,1)(0,0,0)[0] intercept
Total fit time: 4.708 seconds


/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=30957.977, Time=0.06 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=30661.201, Time=0.06 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.54 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=30955.979, Time=0.05 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=30531.598, Time=0.14 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=30473.775, Time=0.21 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=30243.623, Time=0.86 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=30241.832, Time=0.61 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=1.12 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=1.27 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=1.72 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=2.16 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=inf, Time=0.58 sec

Best model:  ARIMA(2,1,1)(0,0,0)[0] intercept
Total fit time: 9.400 seconds


/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


(  Models  Product_Code  Period   Order_Demand
 0  Arima           NaN     NaN  167862.270971,
     Period  Model  Product_Code  Order_Demand
 0      NaN  Arima           NaN           NaN
 1      NaN  Arima           NaN           NaN
 2      NaN  Arima           NaN           NaN
 3      NaN  Arima           NaN           NaN
 4      NaN  Arima           NaN           NaN
 5      NaN  Arima           NaN           NaN
 6      NaN  Arima           NaN           NaN
 7      NaN  Arima           NaN           NaN
 8      NaN  Arima           NaN           NaN
 9      NaN  Arima           NaN           NaN
 10     NaN  Arima           NaN           NaN
 11     NaN  Arima           NaN           NaN)

In [66]:
%%capture
import pandas as pd
# build function to run model for all columns
Prophetf = ['Prophet']
ProphetfFcastPerf = pd.DataFrame({'Models': Prophetf})
ProphetfData = pd.DataFrame({'Period': full_period, 'Model': 'Prophet'})
    

           

In [2]:
def prophetfForecast(data): 
    for i in data.columns:
        try:
            myList = data[i].tolist()
            j = myList.index(next(filter(lambda x: x!=0, myList)))
            temp = data.iloc[j:,]        
            temp['ds']=temp.index.astype('datetime64[ns]')
            temp = temp.reset_index(drop=True)
            temp = temp.rename(columns = {i:'y'})
            temp = temp.loc[:,['ds','y']]
                
            train = temp[:int(2/3*(len(temp)))]
            test = temp[int(2/3*len(temp)):]
               
            # Test model
            model_pred = Prophet()
            model_pred.fit(train)
            pred = model_pred.predict(test)
            ProphetfFcastPerf[i] = sqrt(mean_squared_error(test['y'].values,pred['yhat'].values))
                
            # Forecast model
            model_fc = Prophet()
            model_fc.fit(temp)
                        
            fc_ds = ProphetfData.copy().set_index('Period')
            fc_ds['ds']=fc_ds.index.astype('datetime64[ns]')
            fc_ds = fc_ds.reset_index(drop=True).drop('Model',axis=1)
            forecast = model_fc.predict(fc_ds)
            ProphetfData[i] = np.round(forecast['yhat'].values)
        except:
            ProphetfFcastPerf[i] = np.nan
            ProphetfData[i] = np.nan
    return ProphetfFcastPerf, ProphetfData    

                

In [3]:
prophetfForecast(data)


NameError: name 'data' is not defined